In [54]:
import csv
from konlpy.tag import Okt
from collections import Counter

# 불용어 리스트
stopwords = ['를', '을', '에', '가', '이', '은', '는', '하', '들', '사람', '너', '말', '좀', '걍', '잘', '걔', '한', '되', '음', '것', '과', '수', '다', '뒤', '나', '이다', '내', '그']

# 추출할 품사 리스트 정의
included_pos = ['Noun', 'Verb', 'Adjective', 'Adverb']

okt = Okt()

for i in range(1,7):
    input_file = f"C:/workspace/project_bigdata/data/textbook_data/kor-{i}.csv"
    output_file = f"../data/raw/word/count/textbook/textbook-{i}.csv"
    
    word_counter = Counter()
    # count=0
    
    with open(input_file, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter=',') 
    
        for row in reader:
            if row[1] == '빈도수':
                continue
            
            if len(row) != 2:
                print(f"Unexpected row format: {row}")
                continue
        
            word, freq = row
            freq = int(freq)
            
            # 형태소 분석 (stem=True 옵션으로 어근 추출)
            morphs_with_pos = okt.pos(word, stem=True)
            # print(f"Original word: {word}, Morphs: {morphs_with_pos}")  # 디버깅 용도
            
            # 어근 단어로 빈도수 합산
            for morph, pos in morphs_with_pos:
                if pos in included_pos and morph not in stopwords:
                    word_counter[morph] += freq
                # else:
                #     count=count+1
    
    filtered_word_counter = {word: freq for word, freq in word_counter.items() if freq >= 30}
    
    row=0
    # 결과를 새로운 CSV 파일로 저장
    with open(output_file, 'w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(['Word', 'Count'])
        for i, (word, freq) in enumerate(filtered_word_counter.items(), start=1):
            # print(f"Original word: {word}, Freq: {freq}")  # 디버깅 용도
            writer.writerow([word, freq])
            row=row+1
    
    print(f"어근 단위로 통합된 단어 리스트가 '{output_file}' 파일에 저장되었습니다.")
    print(row)

어근 단위로 통합된 단어 리스트가 '../data/raw/word/count/textbook/textbook-1.csv' 파일에 저장되었습니다.
135
어근 단위로 통합된 단어 리스트가 '../data/raw/word/count/textbook/textbook-2.csv' 파일에 저장되었습니다.
228
어근 단위로 통합된 단어 리스트가 '../data/raw/word/count/textbook/textbook-3.csv' 파일에 저장되었습니다.
346
어근 단위로 통합된 단어 리스트가 '../data/raw/word/count/textbook/textbook-4.csv' 파일에 저장되었습니다.
360
어근 단위로 통합된 단어 리스트가 '../data/raw/word/count/textbook/textbook-5.csv' 파일에 저장되었습니다.
291
어근 단위로 통합된 단어 리스트가 '../data/raw/word/count/textbook/textbook-6.csv' 파일에 저장되었습니다.
398


In [55]:
import os
import json
import csv
from konlpy.tag import Okt
from collections import Counter

okt = Okt()

# JSON 파일을 읽어서 DataFrame으로 변환하는 함수
def json_files_to_word_count(folder_path):
    all_texts = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8-sig') as f:
                try:
                    json_data = json.load(f)
                    # srcText 데이터 추출
                    extracted_texts = extract_columns_from_json(json_data)
                    all_texts.extend(extracted_texts)  # 모든 텍스트를 리스트에 추가
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from file {filename}: {e}")
    
    # 전체 텍스트에 대해 형태소 분석 후 단어 빈도 계산
    word_counter = analyze_texts_and_count_words(all_texts)
    return word_counter

# 특정 컬럼만 추출하기 위한 함수
def extract_columns_from_json(json_data):
    extracted_data = []
    if 'paragraphInfo' in json_data:
        for item in json_data['paragraphInfo']:
            extracted_data.append(item.get("srcText", ""))  # srcText만 추출
    return extracted_data

def analyze_texts_and_count_words(texts):
    word_counter = Counter()
    for text in texts:
        # 형태소 분석을 하고 품사 정보를 같이 반환
        tokens_with_pos = okt.pos(text, norm=True, stem=True)
        # 특정 품사에 해당하는 단어들만 필터링
        filtered_tokens = [word for word, pos in tokens_with_pos if pos in included_pos and word not in stopwords]
        word_counter.update(filtered_tokens)
    return word_counter
    
# 빈도수가 100 이상인 단어들만 CSV 파일로 저장
def save_frequent_words_to_csv(word_counter, output_file, min_count=100):
    row=0           
    with open(output_file, 'w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(['Word', 'Count'])
        for word, count in word_counter.items():
            if count >= min_count:
                print(f"Original word: {word}, Freq: {count}") 
                writer.writerow([word, count])
                row=row+1
    
    print(f"어근 단위로 통합된 단어 리스트가 '{output_file}' 파일에 저장되었습니다.")
    print(row)


keyword = ["preschool", "school_123", "school_456"]

for keyword in keyword:
    # JSON 파일들이 있는 폴더 경로
    folder_path = f"C:/workspace/project_bigdata/data/story_test_data/{keyword}"
    output_file = f"../data/raw/word/count/story/story_{keyword}.csv"
    # 전체 작업 실행
    word_counter = json_files_to_word_count(folder_path)
    save_frequent_words_to_csv(word_counter, output_file)



Original word: 하다, Freq: 757
Original word: 있다, Freq: 462
Original word: 가다, Freq: 104
Original word: 되다, Freq: 108
Original word: 엄마, Freq: 143
Original word: 보다, Freq: 162
Original word: 먹다, Freq: 103
Original word: 만들다, Freq: 103
Original word: 우리, Freq: 103
Original word: 그림, Freq: 158
어근 단위로 통합된 단어 리스트가 '../data/raw/word/count/story/story_preschool.csv' 파일에 저장되었습니다.
10
Original word: 아빠, Freq: 124
Original word: 가다, Freq: 128
Original word: 하다, Freq: 975
Original word: 없다, Freq: 137
Original word: 보다, Freq: 272
Original word: 우리, Freq: 147
Original word: 주다, Freq: 143
Original word: 있다, Freq: 529
Original word: 않다, Freq: 152
Original word: 오다, Freq: 172
Original word: 모두, Freq: 102
Original word: 친구, Freq: 107
Original word: 엄마, Freq: 162
Original word: 안, Freq: 142
Original word: 좋다, Freq: 125
Original word: 그렇다, Freq: 161
Original word: 나다, Freq: 114
Original word: 되다, Freq: 126
Original word: 먹다, Freq: 155
Original word: 마리, Freq: 102
어근 단위로 통합된 단어 리스트가 '../data/raw/word/count/

In [56]:
import pandas as pd

def read_file(filepath):
    return pd.read_csv(filepath, sep=',')

def calculate_points(textbook_file, story_file):
    textbook_df = read_file(textbook_file)
    story_df = read_file(story_file)

    # 단어 기준으로 병합 (outer join으로 공통 단어 + 한쪽만 있는 단어 모두 포함)
    merged_df = pd.merge(textbook_df, story_df, on='Word', how='outer', suffixes=('_textbook', '_story'))
    
    # NaN을 0으로 변환하고, 빈도수 합산
    merged_df.fillna(0, inplace=True)
    merged_df['Point'] = merged_df['Count_textbook']*3 + merged_df['Count_story']
    merged_df.drop(['Count_textbook', 'Count_story'], axis=1, inplace=True)
    print(merged_df)
    return merged_df

def convert_point(story_file):
    story_df = read_file(story_file)
    story_df['Point'] = story_df['Count']
    story_df.drop(['Count'], axis=1, inplace=True)
    print(story_df)
    return story_df

# 키워드와 파일 설정
textbook_files = [f"../data/raw/word/count/textbook/textbook-{i}.csv" for i in range(1, 7)]
story_files = {
    "school_123": "../data/raw/word/count/story/story_school_123.csv",
    "school_456": "../data/raw/word/count/story/story_school_456.csv"
}

merged_df = convert_point("../data/raw/word/count/story/story_preschool.csv")
output_file = f"../data/raw/word/count/combined_{1}.csv"
print(output_file)
merged_df.to_csv(output_file, sep=',', index=False, encoding='utf-8-sig')
for i in range(1, 7):
    textbook_file = f"../data/raw/word/count/textbook/textbook-{i}.csv"
    
    if i <= 3:
        story_file = story_files["school_123"]
    else:
        story_file = story_files["school_456"]
    
    # textbook과 story 파일 병합
    merged_df = calculate_points(textbook_file, story_file)
    output_file = f"../data/raw/word/count/combined_{i+1}.csv"
    print(output_file)
    merged_df.to_csv(output_file, sep=',', index=False, encoding='utf-8-sig')


  Word  Point
0   하다    757
1   있다    462
2   가다    104
3   되다    108
4   엄마    143
5   보다    162
6   먹다    103
7  만들다    103
8   우리    103
9   그림    158
../data/raw/word/count/combined_1.csv
    Word  Point
0     가다  470.0
1     가지  189.0
2     겪다  168.0
3    고르다   93.0
4    고맙다  105.0
..   ...    ...
133   행동  123.0
134   확인  249.0
135   활동  132.0
136   활용  303.0
137   흉내  120.0

[138 rows x 2 columns]
../data/raw/word/count/combined_2.csv
    Word  Point
0     가게   96.0
1     가다  692.0
2    가운데  141.0
3     가족  201.0
4     가지  201.0
..   ...    ...
224    형  120.0
225  호랑이  126.0
226   확인  270.0
227   활동  168.0
228   활용  291.0

[229 rows x 2 columns]
../data/raw/word/count/combined_3.csv
    Word   Point
0     가다  1073.0
1    가운데   213.0
2     가장   198.0
3     가지   276.0
4      간   252.0
..   ...     ...
343  호랑이   141.0
344   확인   219.0
345   활동   207.0
346   활용   252.0
347    후   114.0

[348 rows x 2 columns]
../data/raw/word/count/combined_4.csv
    Word  Point
0     가다  937.0
1 

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

def combine_results(file_pattern, num_files):
    combined_df = pd.DataFrame()
    for i in range(1, num_files + 1):
        file_path = file_pattern.format(i)
        df = pd.read_csv(file_path, sep=',')
        df['Grade'] = str(i)+"레벨"  # Add grade information
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df

def prepare_dtm(combined_df):
    dtm_df = combined_df.pivot_table(index='Word', columns='Grade', values='Point', fill_value=0)
    dtm_df = dtm_df.reset_index()
    return dtm_df

def calculate_tfidf(dtm_df):
    dtm_df = dtm_df.reset_index()  # Reset index to ensure 'Word' is a column
    dtm_df.set_index('Word', inplace=True) 
    # Convert the DataFrame to a format suitable for TF-IDF calculation
    dtm_text = dtm_df.apply(lambda x: ' '.join([f"{col}" for col in x.index if x[col] > 0]), axis=1)
    
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(dtm_text)
    
    # Create DataFrame for TF-IDF scores
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=dtm_df.index, columns=vectorizer.get_feature_names_out())
    return tfidf_df


file_pattern = "../data/raw/word/count/combined_{}.csv"
textbook_df = combine_results(file_pattern, 7)

# Prepare document-term matrix
dtm_df = prepare_dtm(textbook_df)
dtm_df.to_csv("../data/raw/word/count/combined.csv", sep=',', encoding='utf-8-sig')
print(dtm_df)

# Calculate TF-IDF
tfidf_df = calculate_tfidf(dtm_df)

# Save the TF-IDF results
tfidf_df.to_csv("../data/raw/word/count/combined_tfidf.csv", sep=',', encoding='utf-8-sig')
print("TF-IDF analysis completed and saved.")


Grade Word    1레벨    2레벨    3레벨     4레벨    5레벨    6레벨     7레벨
0       가게    0.0    0.0   96.0     0.0    0.0    0.0     0.0
1       가다  104.0  470.0  692.0  1073.0  937.0  940.0  1102.0
2      가운데    0.0    0.0  141.0   213.0  186.0  237.0   135.0
3       가장    0.0    0.0    0.0   198.0  126.0  168.0   123.0
4       가족    0.0    0.0  201.0     0.0   93.0    0.0     0.0
..     ...    ...    ...    ...     ...    ...    ...     ...
677     흉내    0.0  120.0    0.0     0.0    0.0    0.0     0.0
678     흐름    0.0    0.0    0.0     0.0  111.0    0.0     0.0
679      흑    0.0    0.0    0.0     0.0    0.0  105.0     0.0
680      힘    0.0    0.0    0.0     0.0    0.0   90.0     0.0
681    힘들다    0.0    0.0    0.0     0.0    0.0    0.0    99.0

[682 rows x 8 columns]
TF-IDF analysis completed and saved.


In [58]:
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt
# 
# def generate_word_cloud(tfidf_df, grade):
#     # 특정 학년에 대한 TF-IDF 점수 필터링
#     if grade not in tfidf_df.columns:
#         print(f"데이터프레임에 '{grade}' 학년이 없습니다.")
#         return
#     
#     grade_tfidf = tfidf_df[grade]
#     
#     # TF-IDF 점수를 딕셔너리로 변환
#     word_freq = grade_tfidf.to_dict()
#     font_path = 'C:/Windows/Fonts/malgun.ttf'
#     
#     # 워드 클라우드 생성
#     wordcloud = WordCloud(font_path=font_path,width=800, height=400, background_color='white').generate_from_frequencies(word_freq)
#     
#     # 워드 클라우드 표시
#     plt.figure(figsize=(10, 5))
#     plt.imshow(wordcloud, interpolation='bilinear')
#     plt.axis('off')
#     plt.title(f"{grade}의 워드 클라우드")
#     plt.show()
# 
# print("TF-IDF DataFrame 열 이름:", tfidf_df.columns)
# 
# # 각 학년별 워드 클라우드 생성 및 표시
# for grade in range(1, 7):
#     generate_word_cloud(tfidf_df, str(grade)+"레벨")


In [59]:
import pandas as pd

def filter_tfidf(tfidf_df, threshold=0.5):
    results = {}
    excluded_words = set()
    
    for grade in tfidf_df.columns:
        if grade == 'index':
            continue
        
        high_tfidf_words = tfidf_df[tfidf_df[grade] > threshold][[grade]]
        
        high_tfidf_words = high_tfidf_words[~high_tfidf_words.index.isin(excluded_words)]
        
        excluded_words.update(high_tfidf_words.index)
        
        # Store the results
        results[grade] = high_tfidf_words.reset_index()  # Remove 'Word' index
    
    return results

def save_filtered_results(filtered_results):
    for grade, df in filtered_results.items():
        file_path = f"../data/raw/word/filtered_{grade}.csv"
        df.to_csv(file_path, index=False, sep=',', encoding='utf-8-sig')
        print(f"Saved filtered results for {grade} to {file_path}")

tfidf_df = pd.read_csv("../data/raw/word/count/combined_tfidf.csv", sep=',', index_col=0)
filtered_results = filter_tfidf(tfidf_df)
save_filtered_results(filtered_results)

Saved filtered results for 1레벨 to ../data/raw/word/filtered_1레벨.csv
Saved filtered results for 2레벨 to ../data/raw/word/filtered_2레벨.csv
Saved filtered results for 3레벨 to ../data/raw/word/filtered_3레벨.csv
Saved filtered results for 4레벨 to ../data/raw/word/filtered_4레벨.csv
Saved filtered results for 5레벨 to ../data/raw/word/filtered_5레벨.csv
Saved filtered results for 6레벨 to ../data/raw/word/filtered_6레벨.csv
Saved filtered results for 7레벨 to ../data/raw/word/filtered_7레벨.csv


In [62]:
combined_df = pd.DataFrame(columns=['Word', 'Level'])

for i in range(1,8):
    filtered = pd.read_csv(f"../data/raw/word/filtered_{i}레벨.csv", sep=',', index_col=0)
    kice = pd.read_csv(f"../data/raw/word/kice_{i}레벨.csv", encoding='cp949', sep=',')
    
    filtered['Level'] = i
    filtered['Word'] = filtered.index
    kice['Level'] = i
    
    combined = pd.concat([filtered, kice], ignore_index=False)
    combined.reset_index(inplace=True, drop=True)
    combined = combined[['Word', 'Level']]
    # combined_df에 중복되지 않은 단어만 추가
    combined = combined[~combined['Word'].isin(combined_df['Word'])]
    
    combined.sort_values(by='Word').reset_index(drop=True)
    
    
    combined_df = pd.concat([combined_df, combined], ignore_index=True)
    print(combined_df)
    

file_path = f"../data/processed/word_dictionary.csv"
combined_df.to_csv(file_path, index=False, sep=',', encoding='utf-8-sig')
print(f"Saved dictionary to {file_path}")

# 해야할일
# 말도 안되는 단어 거르기 - 사전 API 활용... 귀찮은데...


    Word Level
0     가다     1
1     그림     1
2     되다     1
3    만들다     1
4     먹다     1
..   ...   ...
518  화요일     1
519  화장실     1
520  흔들다     1
521    흙     1
522   흰색     1

[523 rows x 2 columns]
       Word Level
0        가다     1
1        그림     1
2        되다     1
3       만들다     1
4        먹다     1
...     ...   ...
1021     훨훨     2
1022  휴대 전화     2
1023     흉내     2
1024      힘     2
1025    힘들다     2

[1026 rows x 2 columns]
      Word Level
0       가다     1
1       그림     1
2       되다     1
3      만들다     1
4       먹다     1
...    ...   ...
1390    훨씬     3
1391  흩어지다     3
1392    희망     3
1393   힘쓰다     3
1394   힘없이     3

[1395 rows x 2 columns]
      Word Level
0       가다     1
1       그림     1
2       되다     1
3      만들다     1
4       먹다     1
...    ...   ...
1675  흐뭇하다     4
1676   흥겹다     4
1677    흥미     4
1678  흥미롭다     4
1679   힘차다     4

[1680 rows x 2 columns]
      Word Level
0       가다     1
1       그림     1
2       되다     1
3      만들다     1
4       먹다  